# Santander

# 1. Import libs
[top](#Contents)

In [10]:
# pandas and numpy imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.metrics')

In [11]:
# pip install -U imbalanced-learn

# 2. Get data
[top](#Contents)

#### Read data
[top](#Contents)

In [4]:
bank_ds = pd.read_csv('train.csv')

In [12]:
bank_test_ds = pd.read_csv('test.csv')

#### Drop an unneeded column

In [5]:
# Assuming 'column_to_drop' is the name of the column you want to drop
column_to_drop = 'ID_code'

# Drop the column from bank_ds
bank_ds.drop(column_to_drop, axis=1, inplace=True)


### Split 70/30 (stratified sampling)

In [6]:
target = bank_ds['target']

In [7]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    bank_ds.drop(labels=['target'], axis=1),  # drop the target
    bank_ds['target'],  # just the target
    test_size=0.3,
    stratify=target,
    random_state=42)

X_train.shape, X_test.shape

((140000, 200), (60000, 200))

### Scaling

Models like Logistic Regression require scaling for better performance. Tree-based models like Random Forest, XGBoost, and LightGBM do not inherently require scaling but scaling can still be beneficial (can improve performance and convergence in some cases).

In [8]:
from sklearn.preprocessing import MinMaxScaler
# we put the variables in the same scale
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Train ML models
### Random Forests

In [8]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=2, n_jobs=4)

rf.fit(X_train, y_train)

y_train_rf = rf.predict_proba(X_train)[:,1]
y_test_rf = rf.predict_proba(X_test)[:,1]

### Logistic Regression

In [9]:
logit = LogisticRegression(random_state=42,  max_iter=2000)

logit.fit(X_train, y_train)

y_train_logit = logit.predict_proba(X_train)[:,1]
y_test_logit = logit.predict_proba(X_test)[:,1]

### XGB Classifier

In [10]:


xgb_model = XGBClassifier(random_state=42, use_label_encoder=False)

xgb_model.fit(X_train, y_train)

y_train_xgb = xgb_model.predict_proba(X_train)[:, 1]
y_test_xgb = xgb_model.predict_proba(X_test)[:, 1]


### Light GBM

In [11]:


lgb_model = LGBMClassifier(random_state=42)

lgb_model.fit(X_train, y_train)

y_train_lgb = lgb_model.predict_proba(X_train)[:, 1]
y_test_lgb = lgb_model.predict_proba(X_test)[:, 1]


[LightGBM] [Info] Number of positive: 14069, number of negative: 125931
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.257812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100493 -> initscore=-2.191760
[LightGBM] [Info] Start training from score -2.191760


### CatBoost

In [9]:


catboost_model = CatBoostClassifier(random_state=42, verbose=False)  # Disable logging
catboost_model.fit(X_train, y_train)

# Predict probabilities for the training and test sets
y_train_catboost = catboost_model.predict_proba(X_train)[:, 1]
y_test_catboost = catboost_model.predict_proba(X_test)[:, 1]

## Evaluation

#### Metrics which require predict method

##### Confusion matrix
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

TN | FP

FN | TP

#### Confusion matrix, Recall, Precision, F1-score, G-mean, etc 

In [14]:
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# require scaling
from imblearn.metrics import (
    geometric_mean_score,
    make_index_balanced_accuracy,
)

def dominance(y_true, y_pred):
    tpr = recall_score(y_test, y_pred, pos_label=1)
    tnr = recall_score(y_test, y_pred, pos_label=0)
    return tpr - tnr

# Function to get classification metrics
def get_classification_metrics(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    accuracy          = accuracy_score(y_test, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Adding possible averages to recall, precision, and f1-score calculations
    # Default average is None - scores are returned for each class individually rather than being averaged
    recall_weighted    = recall_score(y_test, y_pred, average='weighted')
    precision_weighted = precision_score(y_test, y_pred, average='weighted') 
    f1_weighted        = f1_score(y_test, y_pred, average='weighted')

    recall_micro       = recall_score(y_test, y_pred, average='micro')
    precision_micro    = precision_score(y_test, y_pred, average='micro')
    f1_micro           = f1_score(y_test, y_pred, average='micro')
    
    recall_macro       = recall_score(y_test, y_pred, average='macro')
    precision_macro    = precision_score(y_test, y_pred, average='macro')
    f1_macro           = f1_score(y_test, y_pred, average='macro')

    # Adding possible averages to G-mean
    # If the average parameter is not specified, it defaults to average='binary'.
    g_mean_binary      = geometric_mean_score(y_test, y_pred, average='binary')
    g_mean_weighted    = geometric_mean_score(y_test, y_pred, average='weighted')
    g_mean_micro       = geometric_mean_score(y_test, y_pred, average='micro')
    g_mean_macro       = geometric_mean_score(y_test, y_pred, average='macro')

    # A lower alpha gives more weight to sensitivity (TPR), while a higher alpha gives more weight to specificity (TNR). 
    # In other words,in case of a lower alpha more emphasis is placed on correctly identifying positive instances
    gmean = make_index_balanced_accuracy(alpha=0.5, squared=True)(geometric_mean_score) 
    corrected_g_mean  = gmean(y_test, y_pred)
    # specifying an average might not be necessary or meaningful. Leaving for the time being
    corrected_g_mean_binary  = gmean(y_test, y_pred,average='binary')
    corrected_g_mean_weighted  = gmean(y_test, y_pred,average='weighted')
    corrected_g_mean_micro  = gmean(y_test, y_pred,average='micro')
    corrected_g_mean_macro  = gmean(y_test, y_pred,average='macro')
    
    dominance_score   = dominance(y_test, y_pred) 

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    FPR = fp / (tn + fp)
    FNR = fn / (tp + fn)
    # TPR = tp / (tp + fn) # recallminority
    # TNR = tn / (tn + fp) # recall majority
    
    return {
        'Accuracy': accuracy,
        'Recall Majority (TNR)': report['0']['recall'],
        'Recall Minority (TPR)': report['1']['recall'],
        'Balanced Accuracy': balanced_accuracy,
        'FPR': FPR,
        'FNR': FNR,
        
        'Precision Majority': report['0']['precision'],
        'Precision Minority': report['1']['precision'],
        'F1-Score Majority': report['0']['f1-score'],
        'F1-Score Minority': report['1']['f1-score'], 
        
        # Weighted average (Use 'weighted' if you want to take class imbalance into account)
        'Weighted Precision': precision_weighted,
        'Weighted Recall': recall_weighted,
        'Weighted F1-Score': f1_weighted,

        # Micro average (Use 'micro' if you want a metric that gives equal weight to every individual prediction)
        'Micro Precision': precision_micro,
        'Micro Recall': recall_micro,
        'Micro F1-Score': f1_micro,
        
        # Macro average (Use 'macro' if you want to treat each class equally)
        'Macro Precision': precision_macro,
        'Macro Recall': recall_macro,
        'Macro F1-Score': f1_macro,

        # Geometric average (a suitable metric when you want to balance the trade-off between detecting the minority class 
        # and avoiding false positives from the majority class)
        'G-mean-binary': g_mean_binary, # (compute two geometric means for each class, then the arithmetic mean of these G-means)
        'G-mean-weighted': g_mean_weighted, # calculates the geometric mean for each class and then computes the weighted arithmetic mean of these G-means
        'G-mean-micro': g_mean_micro, # treats all classes equally, regardless of their size
        'G-mean-macro': g_mean_macro, # use when you want to weight each instance equally, regardless of their class

        # Corrected G-means
        'Corrected G-mean': corrected_g_mean,
        'Corrected G-mean-binary': corrected_g_mean_binary,
        'Corrected G-mean-weighted': corrected_g_mean_weighted,
        'Corrected G-mean-micro': corrected_g_mean_micro,
        'Corrected G-mean-macro': corrected_g_mean_macro,
        
        'Dominance': dominance_score
    }

models = {
    'Baseline': y_test_base,
    'Random Forest': rf.predict(X_test),
    'Logistic Regression': logit.predict(X_test),
    'XGBoost': xgb_model.predict(X_test),
    'LightGBM': lgb_model.predict(X_test),
    'CatBoost': catboost_model.predict(X_test)
}

# Collecting results
results = []
confusion_matricies = []
for model_name, y_pred in models.items():
    metrics           = get_classification_metrics(y_test, y_pred)
    conf_matrix       = confusion_matrix(y_test, y_pred)
    results.append({
        'Model': model_name,
        **metrics,
    })
    confusion_matricies.append({
        'Model': model_name,
        'Confusion matrix': conf_matrix
    })

# Create a DataFrame from the results
df_results_1 = pd.DataFrame(results)
df_confusion_matricies = pd.DataFrame(confusion_matricies)

# Print the DataFrame as a table
print(df_results_1)
print(df_confusion_matricies)

                 Model  Accuracy  Recall Majority (TNR)  \
0             Baseline  0.899517               1.000000   
1        Random Forest  0.899517               1.000000   
2  Logistic Regression  0.913517               0.986085   
3              XGBoost  0.913183               0.987660   
4             LightGBM  0.907283               0.998295   

   Recall Minority (TPR)  Balanced Accuracy       FPR       FNR  \
0               0.000000           0.500000  0.000000  1.000000   
1               0.000000           0.500000  0.000000  1.000000   
2               0.263891           0.624988  0.013915  0.736109   
3               0.246475           0.617068  0.012340  0.753525   
4               0.092553           0.545424  0.001705  0.907447   

   Precision Majority  Precision Minority  F1-Score Majority  ...  \
0            0.899517            0.000000           0.947101  ...   
1            0.899517            0.000000           0.947101  ...   
2            0.923029            0.

#### Metrics which require predict_proba method
The ROC AUC score is calculated based on the predicted probabilities for each class, which are obtained using the predict_proba method. This method provides the probability estimates for each class, which are necessary to compute the ROC curve and subsequently the AUC (Area Under the Curve).

In [13]:
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score
)

models = {
    'Baseline': y_test_base,
    'Random Forest': rf.predict_proba(X_test)[:,1],
    'Logistic Regression': logit.predict_proba(X_test)[:,1],
    'XGBoost': xgb_model.predict_proba(X_test)[:,1],
    'LightGBM': lgb_model.predict_proba(X_test)[:,1],
    'CatBoost': catboost_model.predict_proba(X_test)[:,1]
}

# Collecting results
results = []
for model_name, y_pred in models.items():
    roc_auc   = roc_auc_score(y_test, y_pred) 
    pr_auc = average_precision_score(y_test, y_pred)
    results.append({
        'Model': model_name,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    })

# Create a DataFrame from the results
df_results_2 = pd.DataFrame(results)

# Print the DataFrame as a table
print(df_results_2)

                 Model   ROC-AUC    PR-AUC
0             Baseline  0.500000  0.100483
1        Random Forest  0.748215  0.277840
2  Logistic Regression  0.858844  0.498939
3              XGBoost  0.854883  0.488493
4             LightGBM  0.863880  0.511203
5             CatBoost  0.893906  0.592294


### Cost-sensitive approach

In [12]:
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score
)
# Assuming ratio is the ratio of the number of negative samples to positive samples
ratio = (y_train == 0).sum() / (y_train == 1).sum()

# Train the models with cost-sensitive approach
rf = RandomForestClassifier(class_weight='balanced')
logit = LogisticRegression(class_weight='balanced', solver='liblinear')
xgb_model = XGBClassifier(scale_pos_weight=ratio)
lgb_model = LGBMClassifier(class_weight='balanced')
catboost_model = CatBoostClassifier(auto_class_weights='Balanced', verbose=False)

# Fit the models
rf.fit(X_train, y_train)
logit.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train)

# Predictions
models = {
    'Random Forest': rf.predict_proba(X_test)[:, 1],
    'Logistic Regression': logit.predict_proba(X_test)[:, 1],
    'XGBoost': xgb_model.predict_proba(X_test)[:, 1],
    'LightGBM': lgb_model.predict_proba(X_test)[:, 1],
    'CatBoost': catboost_model.predict_proba(X_test)[:, 1]
}

# Collecting results
results = []
for model_name, y_pred in models.items():
    roc_auc = roc_auc_score(y_test, y_pred)
    pr_auc = average_precision_score(y_test, y_pred)
    results.append({
        'Model': model_name,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    })

# Create a DataFrame from the results
df_results_2 = pd.DataFrame(results)

# Print the DataFrame as a table
print(df_results_2)


[LightGBM] [Info] Number of positive: 14069, number of negative: 125931
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.235325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
                 Model   ROC-AUC    PR-AUC
0        Random Forest  0.777911  0.300048
1  Logistic Regression  0.858523  0.498147
2              XGBoost  0.851339  0.480928
3             LightGBM  0.868421  0.520339
4             CatBoost  0.891601  0.587179


### Export classification metrics

In [16]:
# Concatenate df_results_1 and df_results_2 horisontally
df_combined_results = pd.concat([df_results_2, df_results_1], axis=1)

# Export results to CSV files
df_combined_results.to_csv('3_evaluation_results_scaling.csv', index=False)
df_confusion_matricies.to_csv('3_confusion_matricies_scaling.csv', index=False)